In [1]:
!pip install transformers torch accelerate bitsandbytes jiwer datasets peft loralib tqdm pytesseract
!apt-get install tesseract-ocr
!apt-get install tesseract-ocr-eng

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 23.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 117 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-eng set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 117 not upgraded.


In [2]:
import torch
from transformers import ( LlamaForCausalLM, LlamaTokenizer, AutoProcessor, AutoModelForVision2Seq, TrainingArguments, Trainer, BitsAndBytesConfig )
from datasets import Dataset, load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import jiwer
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import json
from glob import glob
import random
import pytesseract

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

from huggingface_hub import login

login(token=HF_TOKEN)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda




# Load Images from the folder

In [5]:
def load_images_from_folder(folder):
    images = []
    image_names = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            img = Image.open(os.path.join(folder, filename)).convert("RGB")
            images.append(img)
            image_names.append(filename)
    return images, image_names

image_folder = "/kaggle/input/dataset/images"
dataset, image_names = load_images_from_folder(image_folder)
print("Dataset loaded")

Dataset loaded


# Generate True text from the images using OCR
takes about 12-15 min to generate text

In [6]:
def generate_ground_truth(images, image_names):
    ground_truth = {}
    for img, name in zip(images, image_names):
        text = pytesseract.image_to_string(img)
        ground_truth[name] = text.strip()
    return ground_truth

ground_truth_data = generate_ground_truth(dataset, image_names)
true_texts = [ground_truth_data[img_name] for img_name in image_names]
print("true_texts generated")

true_texts generated


In [9]:
ground_truth_data['india_news_p000060.jpg']

"India-U.S. Space Cooperation:\nReaching for new frontiers\n\nIn the early 1960's, the United States had offered substantial assistance to India in setting up an\nEquatorial Rocket Launching Station at Thumba (TERLS). Subsequently, India dedicated this facility\nto the United Nations in 1968. Since then, scientists from various countries have launched more\nthan 3000 sounding rockets for research purposes.\n\nDuring 1975-76, under a collaborative bilateral agreement, an experiment, Satellite Instrumental\nTelevision Experiment (SITE) was conducted. Under this agreement, a ULS. satellite, ATS-6, beamed\neducational programs to direct reception television sets to 2400 far flung villages exposing them\nto a new and immensely powerful medium of television.\n\nAnuradha, an Indian experiment for cosmic ray studies was part of NASA's third Spacelab mission.\n\nThe Indian Institute of Geomagnetism (IIG) and Survey of India have made use of data received\nfrom NASA’s MAGSAT Satellite for resear

# Creating and Loading the model "Llama-3.2-11B-Vision"
Loaded a 4-bit quantized llama-3.2-11B-vision model. *Note: Loading the model can take upto 10 minutes.*

In [14]:
def load_model():
    model_name = "meta-llama/Llama-3.2-11B-Vision"

    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",  # normalized float 4
        bnb_4bit_use_double_quant=True
    )

    model = AutoModelForVision2Seq.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        device_map="auto",
        torch_dtype=torch.float16
    )

    processor = AutoProcessor.from_pretrained(model_name)

    return model, processor

In [15]:
model, processor = load_model()

config.json:   0%|          | 0.00/5.03k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/89.4k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

# Extract baseline texts using the model
Took about 5 hours to extract text using the model.

In [16]:
def extract_text(images,model,processor):
    texts = []
    for img in tqdm(images):
        inputs = processor(images=img, return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs,max_length=256,num_beams=5,early_stopping=True)
        text = processor.batch_decode(outputs, skip_special_tokens=True)[0]
        texts.append(text)
    return texts

baseline_texts = extract_text(dataset,model,processor)

100%|██████████| 156/156 [5:11:11<00:00, 119.69s/it]  
